In [1]:
#CLSP-Rmin with back-ordering RH all examples
from gurobipy import *
import xlsxwriter
import time
import math
import random
from function_call import compute_values #from model optimization for RH
from function_call_clsp import compute_values as cpCLSP #from model optimization for CLSP Rmin BO
from data import variables
import json
import numpy as np


#start = time.time()

n = Model("CLSP-RminBO")
n.Params.LogToConsole = 0

#Parameters
d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe = variables()

#realized plan - create empty containers
#for CLSP Rmin BO
Items0 = len(Pro)
Periods0 = len(Pe)
lotsizes0 = np.zeros((Items0, Periods0))
lotsizesR0 = np.zeros((Items0, Periods0))

inventory0 = np.zeros((Items0, Periods0))
inventoryR0 = np.zeros((Items0, Periods0))

backorders0 = np.zeros((Items0, Periods0))

setups0 = np.zeros((Items0, Periods0))
setupsR0 = np.zeros((Items0, Periods0))

#for CLSP wirh RH
Items = len(Pro)
Periods = len(Pe)
lotsizes = np.zeros((Items, Periods))
lotsizesR = np.zeros((Items, Periods))

inventory = np.zeros((Items, Periods))
inventoryR = np.zeros((Items, Periods))

backorders = np.zeros((Items, Periods))

setups = np.zeros((Items, Periods))
setupsR = np.zeros((Items, Periods))

start = 0

with open('data.json', 'r') as fa:
    thisfile=json.load(fa)
O=thisfile[0]["O1"]
realizationO=thisfile[0]["realizationO1"]


#################################### Run optymalization ############################# 
#CLSP Rmin BO

Pe=range(T)    
CostBO=True
Rp=10
[BO0,s0,sR0,
oH, ohR, cC, 
oF, oxprod, fcostr, 
oxrework, oCU, ZZ, cbcost, cbcost2,
yS, yR, s, sR, BO, x, xR] = cpCLSP(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,CostBO,realizationO)
for item in Pro:
    for t in range(start, start+Rp):
        lotsizes0[item, t] = yS[item, t-start]
        lotsizesR0[item, t] = yR[item,t-start].x
        inventory0[item, t] = s[item, t-start]
        inventoryR0[item, t] = sR[item, t-start]
        backorders0[item, t] = BO[item, t-start]
        setups0[item, t] = x[item, t-start].x
        setupsR0[item, t] = xR[item, t-start].x
hcost0 = inventory0.sum(axis=1).dot(h)
hcostR0 = inventoryR0.sum(axis=1).dot(hR)
scost0 = setups0.sum(axis=1).dot(f)
scostR0 = setupsR0.sum(axis=1).dot(f)
bcost0 = backorders0.sum(axis=1).dot(np.array(h)*m)

print("REALIZED PLAN0")
print('Total cost0:', hcost0+hcostR0+scost0+scostR0+bcost0)
print('Holding cost0 (good items):', hcost0, 'Holding cost (defect items):', hcostR0)
print('Setup cost0 (good items):', scost0, 'Setup cost (defect items):', scostR0)
print('Backorder cost0:', bcost0)

print('Production plan0:\n', lotsizes0, '\nProduction plan to rework items:\n', lotsizesR0)
print('Inventory0 (good items):\n', inventory0, '\nInventory (defect items):\n', inventoryR0)
print('Realized backorders0:\n', backorders0)

#rolling horizon starts
print("\nCLSP R min with backorders and rolling horizon step by step\n")
print("\nPlanned periods\n")
T=6    
Pe=range(T)    
Rp=2
#10 periods consist of 5 steps with 2 periods realized; here 4 because python calculates from 0

for i in range(5):

    if i ==0:
        print('step',i+1)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp    
        
        
    
    elif i==3: #and T >len(b) and Rp<len(b):
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        #print(BO0)
        #print(s0)
        #print(sR0)
        #print(b)
        #print(d)
        Pe=range(len(b))
        T=len(b)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp   
        
    elif i==4: #T>len(b) and Rp>len(b)
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        Pe=range(len(b))
        T=len(b)
        Rp=len(b)
        #print(BO0)
        #print(s0)
        #print(sR0)
        #print(b)
        #print(d)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, min(start+Rp, Periods)):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp   
    
    
    
    elif i<3: #here more steps included
        print('step',i+1, 'start', start)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        #print(i)
        #print(len(b))
        #print(b)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2, 
         y, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = y[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp   
        
        
    else:
        print('end')
        

hcost = inventory.sum(axis=1).dot(h)
hcostR = inventoryR.sum(axis=1).dot(hR)
scost = setups.sum(axis=1).dot(f)
scostR = setupsR.sum(axis=1).dot(f)
bcost = backorders.sum(axis=1).dot(np.array(h)*m)

print("REALIZED PLAN")
print('Total cost:', hcost+hcostR+scost+scostR+bcost)
print('Holding cost (good items):', hcost, 'Holding cost (defect items):', hcostR)
print('Setup cost (good items):', scost, 'Setup cost (defect items):', scostR)
print('Backorder cost:', bcost)

print('Production plan:\n', lotsizes, '\nProduction plan to rework items:\n', lotsizesR)
print('Inventory (good items):\n', inventory, '\nInventory (defect items):\n', inventoryR)
print('Realized backorders:\n', backorders)


Restricted license - for non-production use only - expires 2024-10-28
O 0.011
O 0.019
O 0.01
O 0.015
O 0.013
O 0.019
O 0.012
O 0.013
O 0.013
O 0.012
O 0.011
O 0.019
O 0.01
O 0.015
O 0.013
O 0.019
O 0.012
O 0.013
O 0.013
O 0.012
O 0.011
O 0.019
O 0.01
O 0.015
O 0.013
O 0.019
O 0.012
O 0.013
O 0.013
O 0.012
O 0.011
O 0.019
O 0.01
O 0.015
O 0.013
O 0.019
O 0.012
O 0.013
O 0.013
O 0.012
REALIZED PLAN0
Total cost0: 14382.5
Holding cost0 (good items): 7552.0 Holding cost (defect items): 1425.0
Setup cost0 (good items): 3820.0 Setup cost (defect items): 790.0
Backorder cost0: 795.5
Production plan0:
 [[ 258.  246.  406.    0.  513.    0.  763.    0.  626.    0.]
 [ 341.  319.  251.  272.  329.  373.  496.  421.  497.  439.]
 [ 510.  469.  391.  398.  481.  566.  666.  710.  752.  674.]
 [ 508.  476.  413. 1153.  590.  981.    0.  800.    0.  682.]] 
Production plan to rework items:
 [[ 0.  0.  0.  0.  0.  0.  0.  0. 33.  0.]
 [ 0.  0.  0. 15.  0.  0.  0.  0.  0. 31.]
 [ 0.  0.  0. 22.  0.  0.

In [ ]:
from gurobipy import *
import xlsxwriter
import time
import math
import random
from function_call import compute_values
from data import variables
import json
import numpy as np

################ roling horizon with 3 to 4 ############

#start = time.time()

n = Model("CLSP-RminBO")

#Parameters
d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe = variables()

#realized plan - create empty containers
Items = len(Pro)
Periods = len(Pe)
lotsizes = np.zeros((Items, Periods))
lotsizesR = np.zeros((Items, Periods))

inventory = np.zeros((Items, Periods))
inventoryR = np.zeros((Items, Periods))

backorders = np.zeros((Items, Periods))

setups = np.zeros((Items, Periods))
setupsR = np.zeros((Items, Periods))

with open('data.json', 'r') as fa:
    thisfile=json.load(fa)
O=thisfile[0]["O1"]
realizationO=thisfile[0]["realizationO1"]

start=0
#rolling horizon starts
print("\nCLSP R min with backorders and rolling horizon step by step\n")
print("\nPlanned periods\n")
T=4    
Pe=range(T)    
Rp=3

for i in range(3):  
    if i ==0:
        print('step',i+1)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp    
        
    
    elif i==2: #and T >len(b) and Rp<len(b):
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        Pe=range(len(b))
        T=len(b)
        print('step',i+1)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp    
        
    
    elif i==3: #T>len(b) and Rp>len(b)
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        Pe=range(len(b))
        T=len(b)
        Rp=len(b)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp    
            
    elif i<2:
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        print('step',i+1)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp    
        
    else:
        print('end')
        
hcost = inventory.sum(axis=1).dot(h)
hcostR = inventoryR.sum(axis=1).dot(hR)
scost = setups.sum(axis=1).dot(f)
scostR = setupsR.sum(axis=1).dot(f)
bcost = backorders.sum(axis=1).dot(np.array(h)*m)

print("REALIZED PLAN")
print('Total cost:', hcost+hcostR+scost+scostR+bcost)
print('Holding cost (good items):', hcost, 'Holding cost (defect items):', hcostR)
print('Setup cost (good items):', scost, 'Setup cost (defect items):', scostR)
print('Backorder cost:', bcost)

print('Production plan:\n', lotsizes, '\nProduction plan to rework items:\n', lotsizesR)
print('Inventory (good items):\n', inventory, '\nInventory (defect items):\n', inventoryR)
np.set_printoptions(suppress=True)
print('Realized backorders:\n', backorders)        



In [ ]:
from gurobipy import *
import xlsxwriter
import time
import math
import random
from function_call import compute_values
from data import variables
import json
import numpy as np

################ roling horizon with 3 to 6 ############

#start = time.time()

n = Model("CLSP-RminBO")

#Parameters
d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe = variables()


#realized plan - create empty containers
Items = len(Pro)
Periods = len(Pe)
lotsizes = np.zeros((Items, Periods))
lotsizesR = np.zeros((Items, Periods))

inventory = np.zeros((Items, Periods))
inventoryR = np.zeros((Items, Periods))

backorders = np.zeros((Items, Periods))

setups = np.zeros((Items, Periods))
setupsR = np.zeros((Items, Periods))

with open('data.json', 'r') as fa:
    thisfile=json.load(fa)
O=thisfile[0]["O1"]
realizationO=thisfile[0]["realizationO1"]


#rolling horizon starts
print("\nCLSP R min with backorders and rolling horizon step by step\n")
print("\nPlanned periods\n")
start=0
T=6    
Pe=range(T)    
Rp=3

for i in range(3):  
    if i ==0:
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp 
    
    elif i==2: #and T >len(b) and Rp<len(b):
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp 
    
    elif i==3: #T>len(b) and Rp>len(b)
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)

        Pe=range(len(b))
        T=len(b)
        Rp=len(b)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp 
    
    elif i<2:
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp 

    else:
        print('end')
        
hcost = inventory.sum(axis=1).dot(h)
hcostR = inventoryR.sum(axis=1).dot(hR)
scost = setups.sum(axis=1).dot(f)
scostR = setupsR.sum(axis=1).dot(f)
bcost = backorders.sum(axis=1).dot(np.array(h)*m)

print("REALIZED PLAN")
print('Total cost:', hcost+hcostR+scost+scostR+bcost)
print('Holding cost (good items):', hcost, 'Holding cost (defect items):', hcostR)
print('Setup cost (good items):', scost, 'Setup cost (defect items):', scostR)
print('Backorder cost:', bcost)

print('Production plan:\n', lotsizes, '\nProduction plan to rework items:\n', lotsizesR)
print('Inventory (good items):\n', inventory, '\nInventory (defect items):\n', inventoryR)
print('Realized backorders:\n', backorders) 